In [2]:
import numpy as np
import pathlib as pl
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
current_path    = pl.Path.cwd().parent.joinpath('img', '01 - Original')
select_folder   = 1 # 1: train, 2: test
folder_path     = current_path.joinpath('train' if select_folder == 1 else 'test')

images_path     = folder_path.joinpath('ims')
mask_path       = folder_path.joinpath('masks')
images_list     = list(images_path.glob('*.npy'))
mask_list       = list(mask_path.glob('*.npy'))

#create a new folder for the Denoised images
new_folder_path = current_path.parent.joinpath('02 -Denoised').mkdir(exist_ok=True)
output_path     = current_path.parent.joinpath('02 -Denoised')

In [4]:
print(output_path)

d:\Trabajo de Maestria\segmentacion_maestria\img\02 -Denoised


In [5]:
def sorting_key(filepath):
    parts           = filepath.stem.split('_')  # Divide el nombre del archivo en partes
    patient_number  = int(parts[1])  # Extrae el número del paciente
    slice_number    = int(parts[2])  # Extrae el número del slice
    return (patient_number, slice_number)  # Retorna una tupla con ambos números

# Ordena images_list basado en el número del paciente y el número del slice
images_list.sort(key=sorting_key)
mask_list.sort(key=sorting_key)

In [ ]:
def remove_noise(m_percentage, files,value,select_folder,select_item,output_path):
    folder_path = output_path.joinpath('train' if select_folder == 1 else 'test').mkdir(exist_ok=True)
    folder_path = output_path.joinpath('train' if select_folder == 1 else 'test').joinpath('ims' if select_item == 1 else 'masks').mkdir(exist_ok=True)
    output_path = output_path.joinpath('train' if select_folder == 1 else 'test').joinpath('ims' if select_item == 1 else 'masks')
    
    for file in tqdm(files, desc='Denoising images and masks'):
        name = file.stem
        image = np.load(file)
        height_to_remove = int(image.shape[0] * (m_percentage / 100))
        
        mask = np.ones_like(image)
        mask[:height_to_remove] = 0

        image_masked = image.copy()
        image_masked[mask == 0] = value

        np.save(output_path.joinpath(name), image_masked)

# Train Folder

In [ ]:
remove_noise(10, images_list,-1024,1,1,output_path)    # Train , ims

In [ ]:
remove_noise(10, mask_list,0,1,2,output_path)      # Train , masks


# TEST

In [ ]:
remove_noise(10, images_list,-1024,2,1,output_path)    # Train , ims

In [ ]:
remove_noise(10, mask_list,0,2,2,output_path) 

In [ ]:
print(output_path)

In [ ]:
def generar_imagenes_png(select_folder,cmap='gray', output_path=output_path):
    """
    Display images and masks side by side.
    
    Parameters:
    - images_list: List of paths to image files.
    - mask_list: List of paths to mask files.
    - cmap: Color map for displaying images and masks. Default is 'gray'.
    
    Returns:
    - None
    """
    images_path = output_path.joinpath('train' if select_folder == 1 else 'test').joinpath('ims')
    mask_path   = output_path.joinpath('train' if select_folder == 1 else 'test').joinpath('masks')
    images_list = list(images_path.glob('*.npy'))
    mask_list   = list(mask_path.glob('*.npy'))
    temp=200
    for image, mask in tqdm(zip(images_list[:temp], mask_list[:temp]), desc='Displaying images and masks'):
        parts = image.stem.split('_')
        set_folder = parts[0]
        pacient = int(parts[1])
        slice_number = int(parts[2])

        image_data = np.load(image)
        mask_data = np.load(mask)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
        
        ax1.imshow(image_data, cmap=cmap)
        ax1.set_title(f'Imagen: {image.stem}')
        ax1.axis('off')
        
        ax2.imshow(mask_data, cmap=cmap)
        ax2.set_title(f'Máscara: {mask.stem}')
        ax2.axis('off')
        
        fig.suptitle(f'Paciente: {pacient}', fontsize=16)
        
        plt.savefig(output_path.joinpath(f'{image.stem}.png'))
        plt.close()

In [ ]:
generar_imagenes_png(1, cmap='gray', output_path=output_path)

In [ ]:
print(f"The unique values of the mask is: {np.unique(mask_data)}")
print(f"The unique values of the image is: {np.unique(image_data)}")